### Data Processing

In [1]:
import os
import shutil
from pathlib import Path
import mne
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.preprocessing import Normalizer

In [8]:
edf_data_path = Path('./data/edf/train/')
edf_data_path.parts[-1]

'train'

In [2]:
import logging

logger = logging.getLogger(__name__)  # Use the current module's name
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
# formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
# handler.setFormatter(formatter)
logger.addHandler(handler)
# accuracy = 0.95  # Replace with your actual accuracy calculation
logger.info("Current accuracy: %.2f", 2.222)  # Log as info

Current accuracy: 2.22


In [3]:
def positional_encoding(max_length:int, d_model:int, model_type='sinusoidal'):
    """
    Generates positional encodings for a given maximum sequence length and model dimensionality.

    Args:
        max_length (int): The maximum length of the sequence.
        d_model (int): The dimensionality of the model.
        model_type (str): The type of positional encoding to use. Defaults to 'sinusoidal'.

    Returns:
        numpy.ndarray: The positional encoding matrix of shape (max_length, d_model).
    """

    if model_type == 'sinusoidal':
        pe = np.zeros((max_length, d_model))
        position = np.arange(0, max_length, dtype=np.float32).reshape(-1, 1)
        div_term = np.exp(np.arange(0, d_model, 2) * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = np.sin(position * div_term)
        if pe.size % 2 != 0:
            pe[:, 1::2] = np.cos(position[:-1] * div_term)
    else:
        raise ValueError("Unsupported model_type: {}".format(model_type))

    return pe

# pe_train = positional_encoding(X_train.shape[0], X_train.shape[1])
# pe_test = positional_encoding(X_test.shape[0], X_test.shape[1])
# # Add positional encoding to the signal
# X_train =  X_train + pe_train # Add corresponding row of pe matrix
# X_test =  X_test + pe_test

In [4]:
def data_clip(data_path:Path, result_path:Path, data_len:int): # data_path = Path('./data/edf/train/'); 
                                                                # result_path = Path('./data/origin_csv/train/')
    if os.path.exists(result_dir_path):
        shutil.rmtree(result_dir_path)
    os.mkdir(result_dir_path)
    
    
    channels = ['Fp1', 'Fp2', 'F3','F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T3', 'T4',
                'T5', 'T6', 'Fz', 'Cz', 'Pz']
    label_path = data_path
    stage = str(data_path.parts[-1])
    label = pd.DataFrame(columns=['csv_file','label'])
    pe = positional_encoding(data_len, len(channels))    # 1000 needs to adjust according my study
    
    
    for file_path in data_path.glob('**/*.edf'):
        sub_label = str(file_path.parts[3])
        file_name = str(file_path.name).split('.')[0]
    #     print(sub_label, file_path, file_name)
        raw = mne.io.read_raw_edf(file_path)
        freq = raw.info['sfreq']   # 250
    #     logger.info(freq)
        raw.crop(tmin=120)    # start from 120 secs
        n_segments = int(np.floor(raw.times[-1] *freq/data_len))
        pd_frame = raw.to_data_frame(picks=['EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF','EEG F4-REF', 
                                            'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 
                                            'EEG O1-REF', 'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF',
                                            'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF', 'EEG T6-REF', 
                                            'EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF'])
    #     channels = ['EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF','EEG F4-REF', 
    #                                         'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 
    #                                         'EEG O1-REF', 'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF',
    #                                         'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF', 'EEG T6-REF', 
    #                                         'EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF']    
    #     print(pd_frame.shape)
    #     logger.debug(pe)  # Log as info
        for i in range(n_segments):
            start_time = i * data_len  # Start time of the segment in seconds
            end_time = start_time + data_len  # End time of the segment in seconds

            # Extract the segment
            segment = pd_frame.iloc[start_time:end_time, 1:]
    #         logger.info(segment.shape, file_name)

            scaler = Normalizer()
            segment = scaler.fit_transform(segment)
            segment = segment + pe
    #         if segment.shape[0]>1000
            segment = pd.DataFrame(segment, columns=channels)

    #         data, times = raw[:, start_time*freq:end_time*freq]
        #     print(times)
    #         df = raw.to_data_frame()
            segment.to_csv(f'{str(result_dir_path)}/{file_name}_{i+1}.csv', index=False)
    #         np.savetxt(f'./data/origin_csv/train/{file_name}_{i+1}.csv', segment, header=','.join(channels), 
    #                    delimiter=',')
            label.loc[len(label)] = [f'{file_name}_{i+1}.csv', sub_label]
        
        raw.close()
    label.to_csv(f'./data/{stage}_label.csv', index=False)
    

### Process training dataset

In [5]:
# training dataset
edf_data_path = Path('./data/edf/train/')  # need to modify
result_dir_path = Path('./data/origin_csv/train/')
segment_length = 1000
data_clip(edf_data_path, result_dir_path, segment_length)

edf_data_path = Path('./data/edf/eval/')  # need to modify
result_dir_path = Path('./data/origin_csv/eval/')
segment_length = 1000
data_clip(edf_data_path, result_dir_path, segment_length)

Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/train/abnormal/01_tcp_ar/aaaaaool_s004_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/train/abnormal/01_tcp_ar/aaaaalto_s004_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/train/abnormal/01_tcp_ar/aaaaakqa_s002_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/train/abnormal/01_tcp_ar/aaaaaaru_s005_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/train/abnormal/01_tcp_ar/aaaa

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/train/abnormal/01_tcp_ar/aaaaaivl_s002_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/train/abnormal/01_tcp_ar/aaaaamtl_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/train/abnormal/01_tcp_ar/aaaaamwu_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/train/abnormal/01_tcp_ar/aaaaakqu_s003_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /h

Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/train/abnormal/01_tcp_ar/aaaaaiev_s003_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/train/abnormal/01_tcp_ar/aaaaakfj_s002_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/train/abnormal/01_tcp_ar/aaaaalym_s001_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/train/abnormal/01_tcp_ar/aaaaappo_s001_t002.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Document

Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/train/abnormal/01_tcp_ar/aaaaacjh_s003_t002.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/train/abnormal/01_tcp_ar/aaaaaokr_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/train/abnormal/01_tcp_ar/aaaaaopt_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/train/abnormal/01_tcp_ar/aaaaahmd_s002_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/ed

Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/train/abnormal/01_tcp_ar/aaaaapvx_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/train/abnormal/01_tcp_ar/aaaaaiua_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/train/abnormal/01_tcp_ar/aaaaaoiw_s002_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/train/abnormal/01_tcp_ar/aaaaamnd_s002_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/train/abnormal/01_tcp_ar/aaaa

...

Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/eval/abnormal/01_tcp_ar/aaaaaord_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/eval/abnormal/01_tcp_ar/aaaaaohr_s002_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/eval/abnormal/01_tcp_ar/aaaaaoiq_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/eval/abnormal/01_tcp_ar/aaaaanyw_s002_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/eval/abnormal/01_tcp_ar/aaaaagvx_

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/eval/normal/01_tcp_ar/aaaaajiy_s001_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/eval/normal/01_tcp_ar/aaaaairb_s001_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/eval/normal/01_tcp_ar/aaaaaopo_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/eval/normal/01_tcp_ar/aaaaajdz_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Do

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/eval/normal/01_tcp_ar/aaaaaoeg_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/eval/normal/01_tcp_ar/aaaaamfe_s001_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/eval/normal/01_tcp_ar/aaaaanvw_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/eval/normal/01_tcp_ar/aaaaaeab_s003_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Do

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/eval/normal/01_tcp_ar/aaaaakgp_s001_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/eval/normal/01_tcp_ar/aaaaanbq_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/eval/normal/01_tcp_ar/aaaaapoq_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/eval/normal/01_tcp_ar/aaaaanlp_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Do

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/eval/normal/01_tcp_ar/aaaaagfk_s003_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/eval/normal/01_tcp_ar/aaaaaijn_s001_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/eval/normal/01_tcp_ar/aaaaaobl_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Documents/Projects/EEG_transformer/data/edf/eval/normal/01_tcp_ar/aaaaaosw_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/yossi/Do

#### Dataset